# Transformer의 깊은 이해
트랜스포머를 로우레벨부터 구현해보도록 하겠습니다.
트랜스포머를 한층 한층 직접 쌓아보이시면 추후에 트랜스포머 계열들을 쉽게 이해하실 수 있습니다.
나중에 계속 트랜스 포머 계열인 Bert, XLnet, ALBERT, Reformer 등을 공부하시면서 어려움에 봉착할 때마다 만성적으로 고민하며 시간을 날리는 것보다는, 지금 한번 제대로 이해하시면 훗날 크게 도움이 될 것이라 믿습니다.

https://www.tensorflow.org/tutorials/text/transformer 를 쉽게 구현하고 이해할 수 있도록 코드를 약간 수정하였습니다.
포르투갈어를 영어로 번역하는 Seq2Seq 예제를 통해서 트랜스포머에 익숙해져 보기로 하겠습니다.

직관적인 설명은 이 사이트에 더 잘 되어 있으며, 저는 직접 실행해보는 관점에서 설명드리고자 합니다. 시각적 설명을 원하시면 이 사이트에 접속하시기 바랍니다. http://jalammar.github.io/illustrated-transformer/
이번 강의에서는 포르투갈어를 영어로 바꾸는 Seq2Seq 문제를 Transformer를 활용하여 해결해 보도록 하겠습니다.

![Imgur](https://i.imgur.com/Z0QOeAG.png)

# 데이터 로드 
데이터 로드는 tensorflow 2에서 제공하는 tf.data.dataset 형식으로 로드하도록 하겠습니다. dataset 형식으로 데이터를 불러오면, 향후 모델을 훈련시킬 때 @tf.function 데코레이터를 활용하여 상당히 빠르게 학습할 수 있습니다.

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 25 not upgraded.
Need to get 430 MB of archives.
After this operation, 1,392 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 10s (44.4 MB/s)
(Reading database ... 129496 files and directories currently installed.)
Removing libcudnn8-dev (8.1.1.33-1+cuda11.2) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.1.1.33-1+cuda11.2 t

In [ ]:
# 포르투갈어-프랑스어 데이터 로드를 위한 임포트
import tensorflow_datasets as tfds

In [ ]:
# 분석에 필요한 모듈
import os
import pandas as pd
import numpu as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기 
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)

In [ ]:
examples

In [ ]:
type(examples['train'])

In [ ]:
train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples['test']